In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
from datetime import timedelta
import re
import shutil

import glob
%matplotlib inline

import matplotlib as mpl
mpl.use('agg')

# import personal modules
from plotter import plot_mclimate_forecast
import mclimate_funcs as mclim_func
from cw3e_tools import load_GFS_datasets
from build_html_table import create_html_table

In [2]:
%%time

varname = 'ivt'

s = load_GFS_datasets(varname)
forecast = s.calc_vars()

## get month and date from the intialization date
ts = pd.to_datetime(forecast.init_date.values, format="%Y%m%d%H")
mon = ts.strftime('%m')
day = ts.strftime('%d')
print(mon, day)

## load mclimate data
mclimate = mclim_func.load_mclimate(mon, day, varname)

## compare the mclimate to the reforecast
ds = mclim_func.compare_mclimate_to_forecast(forecast, mclimate, varname)

step_lst = ds.step.values

for i, step in enumerate(step_lst):
    print(step)
    plot_mclimate_forecast(ds, forecast, step=step, varname='ivt')

/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'gini' loading failed:
Struct() takes at most 1 argument (3 given)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


07 22
6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
132
138
144
150
156
162
168
174
180
186
192
198
204
210
216
222
228
234
240
CPU times: user 58.8 s, sys: 22.2 s, total: 1min 20s
Wall time: 46.5 s


In [3]:
varname = 'freezing_level'
date_string = ts.strftime('%Y%m%d%H')
s = load_GFS_datasets(varname, fdate=date_string)
forecast = s.calc_vars()

## get month and date from the intialization date
ts = pd.to_datetime(forecast.init_date.values, format="%Y%m%d%H")
mon = ts.strftime('%m')
day = ts.strftime('%d')
print(mon, day)

## load mclimate data
mclimate = mclim_func.load_mclimate(mon, day, 'freezing_level')

## compare the mclimate to the reforecast
ds1 = mclim_func.compare_mclimate_to_forecast(forecast, mclimate, 'freezing_level')

step_lst = ds1.step.values

for i, step in enumerate(step_lst):
    print(step)
    plot_mclimate_forecast(ds1, forecast, step=step, varname='freezing_level')

Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f006.grb.923a8.idx' incompatible with GRIB file
Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f012.grb.923a8.idx' incompatible with GRIB file
Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f018.grb.923a8.idx' incompatible with GRIB file
Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f024.grb.923a8.idx' incompatible with GRIB file
Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f030.grb.923a8.idx' incompatible with GRIB file
Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f036.grb.923a8.idx' incompatible with GRIB file
Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f042.grb.923a8.idx' incompatible with GRIB file
Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f048.grb.923a8.idx' incompatible with GRIB file
Ignoring index file '/home/dnash/comet_data/tmp/gfs_2024072212_f054.grb.923a8.idx' incompatible with GRIB file
I

07 22
6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
132
138
144
150
156
162
168
174
180
186
192
198
204
210
216
222
228
234
240


In [9]:
ds1.max(dim=['lat', 'lon'])['freezing_level'].values

array([0.99, 1.  , 0.99, 0.99, 0.99, 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 0.99, 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 0.99, 1.  , 1.  , 1.  , 0.99, 0.99, 1.  , 0.99, 0.99, 0.99,
       0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.98])

In [14]:
max_val = ds2.max(dim=['lat', 'lon'])

fl_vals = max_val.freezing_level.values*100

In [15]:

fl_str_lst = []
for i, (fl_val, step_val) in enumerate(zip(fl_vals.astype(int), step_lst)):
    fl_str = '{0};{1}'.format(fl_val, step_val)
    fl_str_lst.append(fl_str)
    
fl_str_lst

['99;6',
 '100;12',
 '99;18',
 '99;24',
 '99;30',
 '100;36',
 '100;42',
 '100;48',
 '100;54',
 '100;60',
 '100;66',
 '100;72',
 '100;78',
 '100;84',
 '100;90',
 '99;96',
 '100;102',
 '100;108',
 '100;114',
 '100;120',
 '100;126',
 '100;132',
 '100;138',
 '99;144',
 '100;150',
 '100;156',
 '100;162',
 '99;168',
 '99;174',
 '100;180',
 '99;186',
 '99;192',
 '99;198',
 '99;204',
 '99;210',
 '99;216',
 '99;222',
 '99;228',
 '99;234',
 '98;240']

In [4]:
%%time
ds = ds.rename({'mclimate': 'IVT'})
ds1 = ds1.rename({'mclimate': 'freezing_level'})
ds2 = xr.merge([ds, ds1])

ext_SEAK = [-141., -130., 54.5, 60.] # extent of SEAK 
df = create_html_table(ds2, ext_SEAK)

df



CPU times: user 56.7 ms, sys: 9.51 ms, total: 66.2 ms
Wall time: 67.9 ms


/home/dnash/repos/mclimate_tool_cw3e/build_html_table.py:110: RuntimeWarning: invalid value encountered in cast
  for i, (ivt_val, fl_val, step_val) in enumerate(zip(ivt_vals.astype(int), fl_vals.astype(int), step_lst)):


In [5]:
## write to text file
html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)

# write html to file
text_file = open("out/table_test.html", "w")
text_file.write(html)
text_file.close()

In [6]:
# # Define two pairs of colors (dark and light green/yellow)

# from itertools import cycle

# colors = [(245./255., 240./255., 230./255.), (182./255., 177./255., 169./255.)]  # green, yellow
# color_cycle = cycle(
#     [
#         {
#             k: (int(c[0] * 255), int(c[1] * 255), int(c[2] * 255), a)
#             for k, a in enumerate([1, 0.25])
#         }
#         for c in colors
#     ]
# )

# # Define color for each row

# bg_colors = []

# for i in t.index.get_level_values(0).unique():
#     color = next(color_cycle)
#     row_color = cycle(
#         [
#             {
#                 "props": [("background-color", f"rgba{color[0]}")],
#             },
#             {
#                 "props": [("background-color", f"rgba{color[1]}")],
#             },
#         ]
#     )
#     for _ in range(t.loc[(i,), :].shape[0]):
#         bg_colors.append(next(row_color))
        
# # Style dataframe

# css = [{"selector": f".row{i}"} | color for i, color in enumerate(bg_colors)]

# t.style.set_table_styles(css)

In [7]:
# def get_lat_lons_from_txt_file(textpts_fname):
#     ## read text file with points
#     df = pd.read_csv(textpts_fname, header=None, sep=' ', names=['latitude', 'longitude'], engine='python')
#     df['longitude'] = df['longitude']*-1
#     df = df
#     lons = df['longitude'].values
#     lats = df['latitude'].values

#     x = xr.DataArray(lons, dims=['location'])
#     y = xr.DataArray(lats, dims=['location'])

#     return x, y
# ## select transect points
# x, y = get_lat_lons_from_txt_file('../data/latlon_coast.txt')
# mclimate = mclimate.sel(lon=x, lat=y, method='nearest')
# forecast = forecast.sel(lon=x, lat=y, method='nearest')
# ## apply below loop to transect points only